In [2]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# Step 1: Dataset Splitting
def split_dataset(source_dir, train_dir, validation_dir, test_size=0.2):
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    if not os.path.exists(validation_dir):
        os.makedirs(validation_dir)

    for class_name in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_name)
        if os.path.isdir(class_path):
            images = [img for img in os.listdir(class_path) 
                      if os.path.isfile(os.path.join(class_path, img)) and img.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            if len(images) == 0:  # Handle empty directories
                print(f"Warning: No valid images found in {class_path}. Skipping...")
                continue

            train, val = train_test_split(images, test_size=test_size, random_state=42)

            class_train_dir = os.path.join(train_dir, class_name)
            class_val_dir = os.path.join(validation_dir, class_name)
            os.makedirs(class_train_dir, exist_ok=True)
            os.makedirs(class_val_dir, exist_ok=True)

            for img in train:
                shutil.copy(os.path.join(class_path, img), os.path.join(class_train_dir, img))
            for img in val:
                shutil.copy(os.path.join(class_path, img), os.path.join(class_val_dir, img))

    print("Dataset split completed.")

# Update the paths to match your dataset
source_dir = r"D:\Final Year Project\Banana Dataset"
train_dir = r"D:\Final Year Project\Banana Dataset\train"
validation_dir = r"D:\Final Year Project\Banana Dataset\validation"
split_dataset(source_dir, train_dir, validation_dir)

# Step 2: Data Preparation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Step 3: Model Creation
model = models.Sequential([
    layers.Input(shape=(150, 150, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 4: Model Training
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Save the model
model.save("banana_disease_detection_model.keras")

# Step 5: Testing the Model
from tensorflow.keras.utils import load_img, img_to_array

def predict_image(image_path, model, class_indices):
    img = load_img(image_path, target_size=(150, 150))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    class_labels = {v: k for k, v in class_indices.items()}
    return class_labels[predicted_class]

# Update the path to your test image
image_path = r"D:\Final Year Project\pic.jpg"
predicted_disease = predict_image(image_path, model, train_generator.class_indices)
print(f"Predicted Disease: {predicted_disease}")


Dataset split completed.
Found 2282 images belonging to 8 classes.
Found 574 images belonging to 8 classes.


C:\Users\sfaha\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.3853 - loss: 1.5217 - val_accuracy: 0.5261 - val_loss: 1.1246
Epoch 2/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 63s 880ms/step - accuracy: 0.5455 - loss: 1.0458 - val_accuracy: 0.5854 - val_loss: 0.9693
Epoch 3/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 57s 792ms/step - accuracy: 0.6216 - loss: 0.9518 - val_accuracy: 0.6307 - val_loss: 0.9215
Epoch 4/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 63s 878ms/step - accuracy: 0.6445 - loss: 0.8722 - val_accuracy: 0.6446 - val_loss: 0.9729
Epoch 5/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.6724 - loss: 0.7783 - val_accuracy: 0.6916 - val_loss: 0.7728
Epoch 6/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 66s 917ms/step - accuracy: 0.7184 - loss: 0.7075 - val_accuracy: 0.6411 - val_loss: 0.9135
Epoch 7/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 59s 818ms/step - accuracy: 0.7116 - loss: 0.7174 - val_accuracy: 0.6533 - val_loss: 0.8076
Epoch 8/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.7831 - loss: 0.5741 - val_accuracy: 0.8

In [6]:
# Update the path to your test image
image_path = r"D:\Final Year Project\inn.jpg"
predicted_disease = predict_image(image_path, model, train_generator.class_indices)
print(f"Predicted Disease: {predicted_disease}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicted Disease: Yellow Sigatoka
